In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df=pd.read_csv("/kaggle/input/emotion-detection-from-text/tweet_emotions.csv")
df=df.drop(columns=["tweet_id"])

In [3]:
df

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [4]:
def text_preprocessing(text):
    data=text.lower()
    data_re=re.sub(r'\W+',' ',data)
    data_tokenize=word_tokenize(data_re)
    token=[word for word in data_tokenize if word not in stopwords.words('english')]
    return " ".join(token)

In [5]:
df["content"]=df["content"].apply(text_preprocessing)

In [6]:
tfidf=TfidfVectorizer()
tfidf_data=tfidf.fit_transform(df["content"])
tfidf_data=tfidf_data.toarray()
tfidf_data=pd.DataFrame(tfidf_data,columns=tfidf.get_feature_names_out())

In [7]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["sentiment"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())

In [8]:
tfidf_data.shape

(40000, 48066)

In [9]:
# X_train,X_test,Y_train,Y_test=train_test_split(tfidf_data,one,test_size=0.2,random_state=42)

# # Deep Learning

In [10]:
from keras.layers import Dense,Flatten,BatchNormalization,TimeDistributed,Input
from keras.models import Model
from keras.callbacks import EarlyStopping

In [11]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [12]:
inputs=Input(shape=(48066,))
d=Dense(32,activation="relu")(inputs)
d=Dense(64,activation="relu")(d)
outputs=Dense(13,activation="softmax")(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["accuracy"])

In [13]:
model.fit(tfidf_data,one,batch_size=32,epochs=50,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - accuracy: 0.2494 - loss: 2.1602 - val_accuracy: 0.3524 - val_loss: 1.9759
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.4623 - loss: 1.6154 - val_accuracy: 0.3450 - val_loss: 2.0136
Epoch 3/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.6931 - loss: 1.0232 - val_accuracy: 0.3106 - val_loss: 2.3602
Epoch 4/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.8411 - loss: 0.5492 - val_accuracy: 0.2957 - val_loss: 2.7110
Epoch 5/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.9012 - loss: 0.3393 - val_accuracy: 0.2794 - val_loss: 3.1122
Epoch 6/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.9306 - loss: 0.2328 - val_accuracy: 0.2749 - val_loss: 3.4842
Epoch 6: early stopping


In [15]:
# loss,accuracy=model.evaluate(X_test,Y_test)